In [1]:
import sys; sys.path.append('..')
import prosodic2 as p
from prosodic2 import *
from tqdm import tqdm
import pandas as pd

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_rows=25

In [3]:
txt="""
Shall I compare thee to a summer's day?
Thou art more lovely and more temperate:
Rough winds do shake the darling buds of May,
And summer's lease hath all too short a date;
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimm'd;
And every fair from fair sometime declines,
By chance or nature's changing course untrimm'd;
But thy eternal summer shall not fade,
Nor lose possession of that fair thou ow'st;
Nor shall death brag thou wander'st in his shade,
When in eternal lines to time thou grow'st:
   So long as men can breathe or eyes can see,
   So long lives this, and this gives life to thee.

"""

In [4]:
# txt="""Of man's first disobdience, and the fruit
# Of that forbidden tree"""

In [5]:
# txt="""
# Shall I compare thee to a summer's day?
# Thou art more lovely and more temperate:
# Rough winds do shake the darling buds of May,
# And summer's lease hath all too short a date;
# """

In [6]:
# txt="Into the woods we go"

## Loading text

In [9]:
def load_txt(txt_or_fn,lang=DEFAULT_LANG,progress=True,incl_alt=INCL_ALT,num_proc=DEFAULT_NUM_PROC,**kwargs):
    full_txt=to_txt(txt_or_fn)
    if not full_txt: return

    df=pd.DataFrame()
    objs=[]

    for stanza_i,stanza_txt in enumerate(to_stanzas(full_txt)):
        for line_i,line_txt in enumerate(to_lines(stanza_txt)):
            df=line2df(line_txt,lang=lang,incl_alt=incl_alt,**kwargs)
            df['stanza_i']=stanza_i
            df['line_i']=line_i
            df['line_str']=line_txt
            df['line_ipa']=' '.join(df.query('word_ipa!="" & word_ipa_i==0 & syll_i==0').word_ipa)
            df['line_ii']=list(range(len(df)))
            df['num_syll']=len(df.query('word_ipa!="" & word_ipa_i==0'))
            objs+=[df]
    odf=pd.concat(objs)
    assign_proms(odf)
    
    # add other info
    return setindex(odf)
    
def assign_proms(df):
    # set proms
    df['prom_stress']=pd.to_numeric(df['syll_ipa'].apply(getstress),errors='coerce')
    df['prom_strength']=[
        x
        for i,df_word in df.groupby(['word_i','word_ipa_i'])
        for x in getstrength(df_word)
    ]
    df['is_stressed']=(df['prom_stress']>0).apply(np.int32)
    df['is_unstressed']=1-df['is_stressed']
    df['is_peak']=(df['prom_strength']==True).apply(np.int32)
    df['is_trough']=(df['prom_strength']==False).apply(np.int32)

In [10]:
txtdf = load_txt(txt)
txtdf

is_peak  \
stanza_i line_i line_str                                         line_ipa                                         word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa line_ii            
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0      Shall    0          'ʃæl     0      Shall    'ʃæl     0              1   
                                                                                                                                  1          ʃæl      0      Shall    ʃæl      1              1   
                                                                                                                  1      I        0          'aɪ      0      I        'aɪ      2              1   
                                                                                                                  2      compare  0          kəm.'pɛr 0      com      kəm      3              1   
                                                                                                                                                      1      pare     'pɛr     4              1   
...                                                                                                                                                                                         ...   
         13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 7      gives    0          'gɪvz    0      gives    'gɪvz    7              0   
                                                                                                                  8      life     0          'laɪf    0      life     'laɪf    8              0   
                                                                                                                  9      to       0          tuː      0      to       tuː      9              0   
                                                                                                                  10     thee     0          ðiː      0      thee     ðiː      10             0   
                                                                                                                  11     .        0                   0      .                 11             0   

                                                                                                                                                                                        prom_stress  \
stanza_i line_i line_str                                         line_ipa                                         word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa line_ii                
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0      Shall    0          'ʃæl     0      Shall    'ʃæl     0                1.0   
                                                                                                                                  1          ʃæl      0      Shall    ʃæl      1                0.0   
                                                                                                                  1      I        0          'aɪ      0      I        'aɪ      2                1.0   
                                                                                                                  2      compare  0          kəm.'pɛr 0      com      kəm      3                0.0   
                                                                                                                                                      1      pare     'pɛr     4                1.0   
...                                                                                                                                                                                             ...   
         13     So long lives this, and this gives life 

In [ ]:
# txtdf[txtdf.num_syll==13].xs(5,level='line_i')

In [12]:
# txtdf.num_syll.value_counts()

In [13]:
def iter_lines(txtdf):
    for stanza_i,stanzadf in txtdf.groupby('stanza_i'):
#         lines=tqdm(list(stanzadf.groupby('line_i')),desc='Scanning through lines')
        lines=stanzadf.groupby('line_i')
        for line_,linedf in lines:
            yield linedf

In [15]:
for line_df in iter_lines(txtdf): break #pass
line_df

is_peak  \
stanza_i line_i line_str                                line_ipa                                   word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa line_ii            
0        0      Shall I compare thee to a summer's day? 'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ 0      Shall    0          'ʃæl     0      Shall    'ʃæl     0              1   
                                                                                                                   1          ʃæl      0      Shall    ʃæl      1              1   
                                                                                                   1      I        0          'aɪ      0      I        'aɪ      2              1   
                                                                                                   2      compare  0          kəm.'pɛr 0      com      kəm      3              1   
                                                                                                                                       1      pare     'pɛr     4              1   
                                                                                                   3      thee     0          ðiː      0      thee     ðiː      5              0   
                                                                                                   4      to       0          tuː      0      to       tuː      6              1   
                                                                                                   5      a        0          eɪ       0      a        eɪ       7              1   
                                                                                                   6      summer's 0          'sə.mɛːz 0      sum      'sə      8              1   
                                                                                                                                       1      mer's    mɛːz     9              1   
                                                                                                   7      day      0          'deɪ     0      day      'deɪ     10             1   
                                                                                                   8      ?        0                   0      ?                 11             1   

                                                                                                                                                                         prom_stress  \
stanza_i line_i line_str                                line_ipa                                   word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa line_ii                
0        0      Shall I compare thee to a summer's day? 'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ 0      Shall    0          'ʃæl     0      Shall    'ʃæl     0                1.0   
                                                                                                                   1          ʃæl      0      Shall    ʃæl      1                0.0   
                                                                                                   1      I        0          'aɪ      0      I        'aɪ      2                1.0   
                                                                                                   2      compare  0          kəm.'pɛr 0      com      kəm      3                0.0   
                                                                                                                                       1      pare     'pɛr     4                1.0   
                                                                                                   3      thee     0          ðiː      0      thee     ðiː      5                0.0   
                                                                                                   4      to       0          tuː      0      to       tuː      6               

## Looping windows

In [17]:
def iter_windows(df,window_len=3):
    for linedf in iter_lines(df):
        ldf=linedf.reset_index()
        linedf_nopunc = ldf[ldf.word_ipa!=""]
        linewords=[y for x,y in sorted(linedf_nopunc.groupby('word_i'))]        
        for lwi,lineword_slice in enumerate(slices(linewords,window_len)):
            for wi,word_combo in enumerate(apply_combos(pd.concat(lineword_slice), 'word_i', 'word_ipa_i', combo_key='word_combo_i')):
                word_combo['word_window_i']=lwi
#                 word_combo['word_window_combo_i']=wi
                yield word_combo

In [18]:
for dfw in iter_windows(txtdf): pass#display(dfw)
dfw

,stanza_i,line_i,line_str,line_ipa,word_i,word_str,word_ipa_i,word_ipa,syll_i,syll_str,syll_ipa,line_ii,is_peak,prom_stress,is_unstressed,num_syll,is_stressed,is_trough,prom_strength,word_window_i
8,0,13,"So long lives this, and this gives life to thee.",'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː,8,life,0,'laɪf,0,life,'laɪf,8,0,1.0,0,10,1,0,NaN,7
9,0,13,"So long lives this, and this gives life to thee.",'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː,9,to,0,tuː,0,to,tuː,9,0,0.0,1,10,0,0,NaN,7
10,0,13,"So long lives this, and this gives life to thee.",'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː,10,thee,0,ðiː,0,thee,ðiː,10,0,0.0,1,10,0,0,NaN,7


## Parses

In [19]:
def is_ok_parse(parse,maxS=2,maxW=2):
    return ('s'*(maxS+1)) not in parse and ('w'*(maxW+1)) not in parse

In [20]:
def possible_parses(window_len,maxS=2,maxW=2):
    poss = list(product(*[('w','s') for n in range(window_len)]))
    poss = [''.join(x) for x in poss]
    poss = [x for x in poss if is_ok_parse(x)]
    poss = [x for x in poss if len(x)==window_len]
    return poss

In [21]:
def iter_poss_parses(df_window):
    df=df_window
    num_sylls = len(df.reset_index().query('word_ipa!=""'))    
    for posi,pparse in enumerate(possible_parses(num_sylls)):
        df['parse_i']=posi
        df['parse_ii']=list(range(len(pparse)))
        df['parse']=pparse
        df['syll_parse']=list(pparse)
        df['is_w']=[np.int(x=='w') for x in pparse]
        df['is_s']=[np.int(x=='s') for x in pparse]
        yield setindex(df)

In [22]:
for dfp in iter_poss_parses(dfw): pass
dfp

is_peak  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii            
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8              0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9              0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10             0   

                                                                                                                                                                                                                          is_w  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii         
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8           0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9           0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10          1   

                                                                                                                                                                                                                          prom_stress  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                1.0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9                0.0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10               0.0   

                                                                                                                                                                                                                          is_unstressed  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                  
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                    0   
                                                                      

In [23]:
def parse_group(df,constraints=DEFAULT_CONSTRAINTS):
    dfpc=apply_constraints(df)
    lkeys=[c for c in df.columns if c not in set(dfpc.columns)]
    return df[lkeys].join(dfpc)

In [24]:
parse_group(dfp)

is_peak  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii            
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8              0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9              0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10             0   

                                                                                                                                                                                                                          is_w  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii         
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8           0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9           0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10          1   

                                                                                                                                                                                                                          prom_stress  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                1.0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9                0.0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10               0.0   

                                                                                                                                                                                                                          is_unstressed  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                  
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                    0   
                                                                      

In [25]:
def iter_parsed(df):
    iter=iter_poss_parses(df)
#     iter=tqdm(list(iter))
    for pi,dfp in enumerate(iter):
        dfpc=parse_group(dfp)
        yield dfpc

In [26]:
for dfpc in iter_parsed(dfp): pass
dfpc

word_window_i  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                  
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                    7   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9                    7   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10                   7   

                                                                                                                                                                                                                          is_trough  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii              
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8                0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9                0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10               0   

                                                                                                                                                                                                                          is_peak  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii            
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8              0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9              0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10             0   

                                                                                                                                                                                                                          is_w  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii         
0        13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8           0   
                                                              

## Parallel?

In [27]:
def iter_parsed_windows(df):
    for window in iter_windows(df):
        for parsed in iter_parsed(window):
            yield parsed

In [28]:
for dfpw in iter_parsed_windows(line_df): pass
dfpw

is_peak  \
stanza_i line_i line_str                                line_ipa                                   parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii            
0        0      Shall I compare thee to a summer's day? 'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ 9       ssws  5      a        0          eɪ       0        0      a        eɪ       s          7              1   
                                                                                                                 6      summer's 0          'sə.mɛːz 1        0      sum      'sə      s          8              1   
                                                                                                                                                     2        1      mer's    mɛːz     w          9              1   
                                                                                                                 7      day      0          'deɪ     3        0      day      'deɪ     s          10             1   

                                                                                                                                                                                                           is_w  \
stanza_i line_i line_str                                line_ipa                                   parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii         
0        0      Shall I compare thee to a summer's day? 'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ 9       ssws  5      a        0          eɪ       0        0      a        eɪ       s          7           0   
                                                                                                                 6      summer's 0          'sə.mɛːz 1        0      sum      'sə      s          8           0   
                                                                                                                                                     2        1      mer's    mɛːz     w          9           1   
                                                                                                                 7      day      0          'deɪ     3        0      day      'deɪ     s          10          0   

                                                                                                                                                                                                           prom_stress  \
stanza_i line_i line_str                                line_ipa                                   parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii                
0        0      Shall I compare thee to a summer's day? 'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ 9       ssws  5      a        0          eɪ       0        0      a        eɪ       s          7                0.0   
                                                                                                                 6      summer's 0          'sə.mɛːz 1        0      sum      'sə      s          8                1.0   
                                                                                                                                                     2        1      mer's    mɛːz     w          9                0.0   
                                                                                                                 7      day      0          'deɪ     3        0      day      'deɪ     s          10               1.0   

                                                                                                                                                                                                           is_unstressed  \
stanza_i line_i line_str                                line_ipa                                   parse_i parse word_i wo

In [29]:
def get_parsed_windows(txtdf):
    # get all combos
    return pmap_groups(
        iter_parsed_windows,
        txtdf.groupby(['stanza_i','line_i']),
        progress=True,
        num_proc=7,
        desc='Parsing all windows'
    )

In [30]:
df_parsed_windows = get_parsed_windows(txtdf)
df_parsed_windows

Parsing all windows [x7]: 100%|██████████| 14/14 [00:03<00:00,  3.59it/s]


is_peak  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii            
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0       wwsw  0      Shall    0          'ʃæl     0        0      Shall    'ʃæl     w          0              1   
                                                                                                                                1      I        0          'aɪ      1        0      I        'aɪ      w          2              1   
                                                                                                                                2      compare  0          kəm.'pɛr 2        0      com      kəm      s          3              1   
                                                                                                                                                                    3        1      pare     'pɛr     w          4              1   
                                                                                                                  1       wwss  0      Shall    0          'ʃæl     0        0      Shall    'ʃæl     w          0              1   
...                                                                                                                                                                                                                           ...   
         13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 4       sws   9      to       0          tuː      1        0      to       tuː      w          9              0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      s          10             0   
                                                                                                                  5       ssw   8      life     0          'laɪf    0        0      life     'laɪf    s          8              0   
                                                                                                                                9      to       0          tuː      1        0      to       tuː      s          9              0   
                                                                                                                                10     thee     0          ðiː      2        0      thee     ðiː      w          10             0   

                                                                                                                                                                                                                          is_w  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse word_i word_str word_ipa_i word_ipa parse_ii syll_i syll_str syll_ipa syll_parse line_ii         
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0       wwsw  0      Shall    0          'ʃæl     0        0      Shall    'ʃæl     w          0           1   
                                                                                                                                1      I        0          'aɪ      1        0      I        'aɪ      w          2           1   
                                                                                                                                2      compare  0          kəm.'pɛr 2        0      com      kəm      s          3           0   
                                                                                                              

## Reconstructing lines

In [65]:
def summarize_by_syll_pos(df_parsed_windows_of_line,num_syll=10000):
    df=df_parsed_windows_of_line
    
#     nsyll=num_syll
#     df=df[df.index.get_level_values('line_ii')<nsyll]    
    qcols=['stanza_i','line_i','line_ii','syll_parse']
    dfr=df[[x for x in df.columns if not x in df.index.names]].reset_index()
    icols=[x for x in dfr.columns if x.endswith('_i') or x.endswith('_ii')]
    dfr_i=dfr[set(icols) | set(qcols)]
    dfr_x=dfr[(set(dfr.columns) - set(icols)) | set(qcols)]
    odf_x=dfr_x.groupby(qcols).mean()
    odf_i=dfr_i.groupby(qcols).count()
    odf=odf_x#.join(odf_i)
    return odf.reset_index()



def apply_combos_meter(dfsyll,group1='line_ii',group2='syll_parse',combo_key='line_parse_i',num_syll=None):
    # combo of indices?
    df=dfsyll

    # ok
    combo_opts = [
        [(mtr,mdf.mean()) for mtr,mdf in sorted(df.groupby(group2))]
        for i,grp in sorted(df.groupby(group1))
    ]

    # is valid?
    combos = product(*combo_opts)
    vci=-1
    done=set()
    for ci,combo in enumerate(combos):
        cparse = ''.join(i for i,x in combo)
#         print(cparse,len(cparse),num_syll)
#         if len(cparse)!=num_syll: continue
        cparse=cparse[:num_syll]
        if cparse in done: continue
        if not is_ok_parse(cparse): continue
        vci+=1
        odf=pd.DataFrame(x for i,x in combo)
        odf['parse_i']=vci
        odf['parse']=cparse
#         display(odf)
#         stop
        if type(odf)!=pd.DataFrame or not len(odf): continue
        yield odf
        done|={cparse}

def iter_parsed_lines(df_parsed_windows_of_line):
    numsyll=df_parsed_windows_of_line.num_syll.iloc[0]
    dfsyll = summarize_by_syll_pos(df_parsed_windows_of_line,num_syll=numsyll)
    iterr = apply_combos_meter(dfsyll,num_syll=numsyll)
#     iterr = tqdm(iterr)
    yield from iterr

def get_parsed_lines(dfpw,summarized=True):
    df=pmap_groups(
        iter_parsed_lines,
        dfpw.groupby(level=['stanza_i','line_i']),
        num_proc=1,
        progress=True
    )
    
    badcols = [c for c in df.columns if c.startswith('word_') or c.startswith('syll_') or c=='line_ii']
    df=df[set(df.columns) - set(badcols)]
    qkey=['stanza_i','line_i','parse_i','parse']
    df=df.groupby(qkey).mean().sort_index()
    df=df.sort_values(['stanza_i','line_i','*total'])
    return df

In [66]:
dfparsedlines = get_parsed_lines(df_parsed_windows)
dfparsedlines

Mapping iter_parsed_lines [x1]: 100%|██████████| 14/14 [00:05<00:00,  2.58it/s]


*s/unstressed   is_peak  *w/stressed  \
stanza_i line_i parse_i parse                                              
0        0      0       sswsswssws       0.396694  0.909091     0.123967   
                2       sswsswswss       0.396694  0.909091     0.123967   
                8       sswswsswss       0.396694  0.909091     0.123967   
                34      swsswsswss       0.396694  0.909091     0.123967   
                1       sswsswssww       0.347107  0.909091     0.165289   
...                                           ...       ...          ...   
         13     173     wwswwswssw       0.200000  0.000000     0.300000   
                174     wwswwswsws       0.200000  0.000000     0.300000   
                175     wwswwswsww       0.150000  0.000000     0.350000   
                176     wwswwswwss       0.200000  0.000000     0.300000   
                177     wwswwswwsw       0.150000  0.000000     0.350000   

                                        is_w  prom_stress   *w/peak  num_syll  \
stanza_i line_i parse_i parse                                                   
0        0      0       sswsswssws  0.272727     0.454545  0.247934      10.0   
                2       sswsswswss  0.272727     0.454545  0.247934      10.0   
                8       sswswsswss  0.272727     0.454545  0.247934      10.0   
                34      swsswsswss  0.272727     0.454545  0.247934      10.0   
                1       sswsswssww  0.363636     0.454545  0.330579      10.0   
...                                      ...          ...       ...       ...   
         13     173     wwswwswssw  0.600000     0.500000  0.000000      10.0   
                174     wwswwswsws  0.600000     0.500000  0.000000      10.0   
                175     wwswwswsww  0.700000     0.500000  0.000000      10.0   
                176     wwswwswwss  0.600000     0.500000  0.000000      10.0   
                177     wwswwswwsw  0.700000     0.500000  0.000000      10.0   

                                    is_stressed  is_trough    *total  \
stanza_i line_i parse_i parse                                          
0        0      0       sswsswssws     0.454545   0.090909  0.768595   
                2       sswsswswss     0.454545   0.090909  0.768595   
                8       sswswsswss     0.454545   0.090909  0.768595   
                34      swsswsswss     0.454545   0.090909  0.768595   
                1       sswsswssww     0.454545   0.090909  0.842975   
...                                         ...        ...       ...   
         13     173     wwswwswssw     0.500000   0.000000  0.500000   
                174     wwswwswsws     0.500000   0.000000  0.500000   
                175     wwswwswsww     0.500000   0.000000  0.500000   
                176     wwswwswwss     0.500000   0.000000  0.500000   
                177     wwswwswwsw     0.500000   0.000000  0.500000   

                                    prom_strength      is_s  is_unstressed  
stanza_i line_i parse_i parse                                               
0        0      0       sswsswssws       0.909091  0.727273       0.545455  
                2       sswsswswss       0.909091  0.727273       0.545455  
                8       sswswsswss       0.909091  0.727273       0.545455  
                34      swsswsswss       0.909091  0.727273       0.545455  
                1       sswsswssww       0.909091  0.636364       0.545455  
...                                           ...       ...            ...  
         13     173     wwswwswssw            NaN  0.400000       0.500000  
                174     wwswwswsws            NaN  0.400000       0.500000  
                175     wwswwswsww            NaN  0.300000       0.500000  
                176     wwswwswwss            NaN  0.400000       0.500000  
                177     wwswwswwsw            NaN  0.300000       0.500000  

[2754 rows x 13 columns]

In [68]:
Counter(len(x) for x in dfparsedlines.index.get_level_values('parse'))

Counter({10: 1780, 9: 110, 11: 864})

In [71]:
def get_info_byline(txtdf):
    linedf=txtdf.xs([0,0,0],level=['word_i','word_ipa_i','syll_i'])
    linedf=linedf.reset_index(level=['word_str','word_ipa','syll_str','syll_ipa','line_ii'],drop=True)
    return linedf#.reset_index().set_index(['stanza_i','line_i'])

In [72]:
linedf=get_info_byline(txtdf)
linedf

is_peak  \
stanza_i line_i line_str                                          line_ipa                                                      
0        0      Shall I compare thee to a summer's day?           'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ                1   
         1      Thou art more lovely and more temperate:          'ðaʊ 'ɑrt 'mɔːr 'lʌv.liː ænd 'mɔːr 'tɛm.prət              1   
         2      Rough winds do shake the darling buds of May,     'rəf 'wɪndz 'duː 'ʃeɪk ðə 'dɑr.lɪŋ 'bədz ʌv 'meɪ          0   
         3      And summer's lease hath all too short a date;     'ænd 'sə.mɛːz 'liːs 'hæθ 'ɔːl 'tuː 'ʃɔːrt eɪ 'deɪt        1   
         4      Sometime too hot the eye of heaven shines,        'səm.`taɪm 'tuː 'hɑt ðə 'aɪ ʌv 'hɛ.vən 'ʃaɪnz             0   
         5      And often is his gold complexion dimm'd;          'ænd 'ɔː.fʌn ɪz hɪz 'goʊld kəm.'pɛk.ʃən 'dɪm.`di          0   
         6      And every fair from fair sometime declines,       'ænd 'ɛ.vɛː.iː 'fɛr frʌm 'fɛr 'səm.`taɪm dɪ.'kl...        0   
         7      By chance or nature's changing course untrimm'd;  'baɪ 'ʧæns ɔːr 'neɪ.ʧɛːz 'ʧeɪn.ʤɪŋ 'kɔːrs ʌn.'t...        0   
         8      But thy eternal summer shall not fade,            'bət ðaɪ ɪ.'tɛː.nəl 'sə.mɛː 'ʃæl 'nɑt 'feɪd               0   
         9      Nor lose possession of that fair thou ow'st;      'nɔːr 'luːz pə.'zɛ.ʃən ʌv 'ðæt 'fɛr ðaʊ 'əʊst             0   
         10     Nor shall death brag thou wander'st in his shade, 'nɔːr 'ʃæl 'dɛθ 'bræg ðaʊ 'wɑn.dʌst ɪn hɪz 'ʃeɪd          1   
         11     When in eternal lines to time thou grow'st:       'wɛn ɪn ɪ.'tɛː.nəl 'laɪnz tuː 'taɪm ðaʊ 'ɡɹəʊst           0   
         12     So long as men can breathe or eyes can see,       'soʊ 'lɔːŋ æz 'mɛn kæn 'briːð ɔːr 'aɪz kæn 'siː           0   
         13     So long lives this, and this gives life to thee.  'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː          0   

                                                                                                                      prom_stress  \
stanza_i line_i line_str                                          line_ipa                                                          
0        0      Shall I compare thee to a summer's day?           'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ                  1.0   
         1      Thou art more lovely and more temperate:          'ðaʊ 'ɑrt 'mɔːr 'lʌv.liː ænd 'mɔːr 'tɛm.prət                1.0   
         2      Rough winds do shake the darling buds of May,     'rəf 'wɪndz 'duː 'ʃeɪk ðə 'dɑr.lɪŋ 'bədz ʌv 'meɪ            1.0   
         3      And summer's lease hath all too short a date;     'ænd 'sə.mɛːz 'liːs 'hæθ 'ɔːl 'tuː 'ʃɔːrt eɪ 'deɪt          1.0   
         4      Sometime too hot the eye of heaven shines,        'səm.`taɪm 'tuː 'hɑt ðə 'aɪ ʌv 'hɛ.vən 'ʃaɪnz               1.0   
         5      And often is his gold complexion dimm'd;          'ænd 'ɔː.fʌn ɪz hɪz 'goʊld kəm.'pɛk.ʃən 'dɪm.`di            1.0   
         6      And every fair from fair sometime declines,       'ænd 'ɛ.vɛː.iː 'fɛr frʌm 'fɛr 'səm.`taɪm dɪ.'kl...          1.0   
         7      By chance or nature's changing course untrimm'd;  'baɪ 'ʧæns ɔːr 'neɪ.ʧɛːz 'ʧeɪn.ʤɪŋ 'kɔːrs ʌn.'t...          1.0   
         8      But thy eternal summer shall not fade,            'bət ðaɪ ɪ.'tɛː.nəl 'sə.mɛː 'ʃæl 'nɑt 'feɪd                 1.0   
         9      Nor lose possession of that fair thou ow'st;      'nɔːr 'luːz pə.'zɛ.ʃən ʌv 'ðæt 'fɛr ðaʊ 'əʊst               1.0   
         10     Nor shall death brag thou wander'st in his shade, 'nɔːr 'ʃæl 'dɛθ 'bræg ðaʊ 'wɑn.dʌst ɪn hɪz 'ʃeɪd            1.0   
         11     When in eternal lines to time thou grow'st:       'wɛn ɪn ɪ.'tɛː.nəl 'laɪnz tuː 'taɪm ðaʊ 'ɡɹəʊst             1.0   
         12     So long as men can breathe or eyes can see,       'soʊ 'lɔːŋ æz 'mɛn kæn 'briːð ɔːr 'aɪz kæn 'siː             1.0   
         13     So long lives this, and this gives life to

In [73]:
def joindfs(a,b):
    bcols = set(b.columns) - set(a.columns)
    return a.join(b[bcols])

In [75]:
setindex(joindfs(dfparsedlines, linedf))

*s/unstressed  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse                       
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0       sswsswssws       0.396694   
                                                                                                                  1       sswsswssww       0.347107   
                                                                                                                  2       sswsswswss       0.396694   
                                                                                                                  3       sswsswswsw       0.347107   
                                                                                                                  4       sswsswswws       0.347107   
...                                                                                                                                             ...   
         13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 173     wwswwswssw       0.200000   
                                                                                                                  174     wwswwswsws       0.200000   
                                                                                                                  175     wwswwswsww       0.150000   
                                                                                                                  176     wwswwswwss       0.200000   
                                                                                                                  177     wwswwswwsw       0.150000   

                                                                                                                                      is_trough  \
stanza_i line_i line_str                                         line_ipa                                         parse_i parse                   
0        0      Shall I compare thee to a summer's day?          'ʃæl 'aɪ kəm.'pɛr ðiː tuː eɪ 'sə.mɛːz 'deɪ       0       sswsswssws   0.090909   
                                                                                                                  1       sswsswssww   0.090909   
                                                                                                                  2       sswsswswss   0.090909   
                                                                                                                  3       sswsswswsw   0.090909   
                                                                                                                  4       sswsswswws   0.090909   
...                                                                                                                                         ...   
         13     So long lives this, and this gives life to thee. 'soʊ 'lɔːŋ 'lɪvz ðɪs ænd ðɪs 'gɪvz 'laɪf tuː ðiː 173     wwswwswssw   0.000000   
                                                                                                                  174     wwswwswsws   0.000000   
                                                                                                                  175     wwswwswsww   0.000000   
                                                                                                                  176     wwswwswwss   0.000000   
                                                                                                                  177     wwswwswwsw   0.000000   

                                                                                                                                        *total  \
stanza_i line_i line_str                                         line_ipa                                       

In [ ]:
linemeta=get_line_meta(txtdf)
dfpll=dfparsedlines.reset_index().set_index(['stanza_i','line_i'])
dfpll

In [ ]:
dfalljoined=linemeta.join(dfpll[list(set(dfpll.columns) - set(linemeta.columns))])
dfalljoined=setindex(dfalljoined)
dfalljoined

In [ ]:
dfalljoined[dfalljoined.num_syll==15]